In [1]:
import os
import sys
# Beispiel: setze Projekt-Root manuell (Pfad anpassen)
project_root = os.path.abspath('..')  # oder '..' oder '../..' je nach Notebook-Ordner

import json
import joblib
import matplotlib.pyplot as plt
import numpy as np
from numpy import random
import pandas as pd
from IPython import display  # WICHTIG für Jupyter Live-Update
from pandasgui import show
from tqdm import tqdm
import seaborn as sns
from pprint import pprint
import pickle
import importlib

import src.execution
import src.model
import src.util

importlib.reload(src.execution)
importlib.reload(src.model)
importlib.reload(src.util)

from src.data import NPY
from src.execution import CNNValidationExecution, WindowSplittingExecution, CNNTuningExecution, TrAdaBoostR2TrainingExecution
from src.model import CNN, build_model_for_TrAdaBoostR2, tradaBoostR2_setup
from src.util import timestamp
import importlib

### TradaboostR2 Modells ###

TrAdaBoostR2 Modell mit drei Ausgängen auf den einzelnen Objekten aufbauen.

In [ ]:
dateipfad_femDaten="build\\window_split\\sim_data_preprocessed_offset_multiplitiv\\1744056915\\"
dateipfad_realDaten="build\\window_split\\real-data\\1744062620\\"
model_file = "assets\\models\\untrained\\synthetic-model\\syn-model.h5"
best_learning_rate=0.0005026453831067469
save_folder = os.path.join("build", "tradaboost_model", "training_30_TEST_02")
save_filename="training_09"

modell=tradaBoostR2_setup.TrAdaBoostR2ModelSetup(
    model_path=model_file,
    early_stoppping_model=True,
    patience_model=2,
    learningrate_model=best_learning_rate,
    n_estimators_tradaBoostR2=2,
    save_folder=save_folder,
    epochs_model=1,  # Wert bitte setzen
    batch_size_model=32,  # Wert bitte setzen
    early_stoppping_TraDaBoostR2=False,  # Wert bitte setzen
    learningrate_TraDaBoostR2=1  # Wert bitte setzen
)

training=TrAdaBoostR2TrainingExecution(tradaboostr2_model=modell,
                                                dateipfad_source_daten=dateipfad_femDaten,
                                                dateipfad_target_daten=dateipfad_realDaten,
                                                use_scaled_labels=True,
                                                model_file=model_file,
                                                save_folder=save_folder,
                                                save_filename="testmodel",
                                                training_dokumentation=True)


### Containern 3 Modelle ###

Momentan wird ein Modell mit drei ausgängen verwendet
Weiterentwicklung: Drei Modelle, bei denen jweils der input gleich ist. 

In [3]:
SPLITTED_DATA_FOLDER = "build\\window_split\\sim_data_preprocessed\\1743966827\\"
CNNTuningExecution.execute_tuning_three_model(SPLITTED_DATA_FOLDER,max_trials=3,train_on_scled_labels=True)

Trial 3 Complete [00h 00m 14s]
val_mae: 0.017685137689113617

Best val_mae So Far: 0.017685137689113617
Total elapsed time: 00h 00m 49s
Anzahl an Conv Layers: 1
Kernel Size 0 ist: 5
Anzahl an Fully Connected Layers: 1
Nach dem Modellbau
Output-Namen: ListWrapper(['Verstellweg_Phi'])
Loss dict: mean_absolute_error
Metrics dict: ['loss', 'compile_metrics']

 Modell für   Verstellweg_Phi gespeichert:
- Hyperparameter: build\tune\1743966827\1751560585\Verstellweg_Phi\best-hyperparameters.json
- Modell:         build\tune\1743966827\1751560585\Verstellweg_Phi\best-model.h5
